<a href="https://colab.research.google.com/github/Lakshmiabbaraju/Crowdfunding/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install findspark

In [2]:
# Import findspark and initialize.
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the DataFrame
df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT YEAR(DATE) as Year_Sold_4B, ROUND(AVG(price),2) AS Avg_Price FROM home_sales WHERE BEDROOMS = 4 GROUP BY 1 ORDER BY 1 ASC""").show()


+------------+---------+
|Year_Sold_4B|Avg_Price|
+------------+---------+
|        2019| 300263.7|
|        2020|298353.78|
|        2021|301819.44|
|        2022|296363.88|
+------------+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT date_built AS Home_Built_Year, ROUND(AVG(price),2) AS Avg_Price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY 1 ORDER BY 1 ASC""").show()


+---------------+---------+
|Home_Built_Year|Avg_Price|
+---------------+---------+
|           2010|292859.62|
|           2011|291117.47|
|           2012|293683.19|
|           2013|295962.27|
|           2014|290852.27|
|           2015| 288770.3|
|           2016|290555.07|
|           2017|292676.79|
+---------------+---------+



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT date_built AS Home_Built_Year, ROUND(AVG(price),2)AS Avg_Price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000 GROUP BY 1 ORDER BY 1 ASC""").show()


+---------------+---------+
|Home_Built_Year|Avg_Price|
+---------------+---------+
|           2010|280447.23|
|           2011|281413.45|
|           2012|295858.68|
|           2013|295142.13|
|           2014|294195.13|
|           2015|291788.36|
|           2016| 287812.6|
|           2017|282026.59|
+---------------+---------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Get the current time
start_time = time.time()

# The code below displays the single digit 9 as part of the list
# spark.sql("SELECT view AS Home_View_Rating, ROUND(AVG(price),2) AS Avg_Price FROM home_sales WHERE price>=350000 GROUP BY 1 ORDER BY 1 DESC").show()

# This line of code runs the query and sorts correctly by left-padding the values with '0' so that number 9 is considered 09, and be sorted correctly.
spark.sql("""SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE price >= 350000 GROUP BY 1 ORDER BY Home_View_Rating DESC""").show()

# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----------------+----------+
|Home_View_Rating| Avg_Price|
+----------------+----------+
|              99|1061201.42|
|              98|1053739.33|
|              97|1129040.15|
|              96|1017815.92|
|              95| 1054325.6|
|              94| 1033536.2|
|              93|1026006.06|
|              92| 970402.55|
|              91|1137372.73|
|              90|1062654.16|
|              89|1107839.15|
|              88|1031719.35|
|              87| 1072285.2|
|              86|1070444.25|
|              85|1056336.74|
|              84|1117233.13|
|              83|1033965.93|
|              82| 1063498.0|
|              81|1053472.79|
|              80| 991767.38|
+----------------+----------+
only showing top 20 rows

--- 1.9893498420715332 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# Get the current time
start_time = time.time()

# Repeating the same query as for the previous table
spark.sql("""SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE price >= 350000 GROUP BY 1 ORDER BY Home_View_Rating DESC""").show()

# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+----------------+----------+
|Home_View_Rating| Avg_Price|
+----------------+----------+
|              99|1061201.42|
|              98|1053739.33|
|              97|1129040.15|
|              96|1017815.92|
|              95| 1054325.6|
|              94| 1033536.2|
|              93|1026006.06|
|              92| 970402.55|
|              91|1137372.73|
|              90|1062654.16|
|              89|1107839.15|
|              88|1031719.35|
|              87| 1072285.2|
|              86|1070444.25|
|              85|1056336.74|
|              84|1117233.13|
|              83|1033965.93|
|              82| 1063498.0|
|              81|1053472.79|
|              80| 991767.38|
+----------------+----------+
only showing top 20 rows

--- 0.8387024402618408 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("parket_delayed_partitioned")

In [14]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('parket_delayed_partitioned')

In [15]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_delays_p')

In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

# Get the current time
start_time = time.time()

# Query that calculates the avg price of a home per view rating using the parquet DataFrame
spark.sql("""SELECT LPAD(view, 2, '0') AS Home_View_Rating, ROUND(AVG(price), 2) AS Avg_Price FROM p_delays_p WHERE price >= 350000 GROUP BY 1 ORDER BY Home_View_Rating DESC""").show()

# Print out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----------------+----------+
|Home_View_Rating| Avg_Price|
+----------------+----------+
|              99|1061201.42|
|              98|1053739.33|
|              97|1129040.15|
|              96|1017815.92|
|              95| 1054325.6|
|              94| 1033536.2|
|              93|1026006.06|
|              92| 970402.55|
|              91|1137372.73|
|              90|1062654.16|
|              89|1107839.15|
|              88|1031719.35|
|              87| 1072285.2|
|              86|1070444.25|
|              85|1056336.74|
|              84|1117233.13|
|              83|1033965.93|
|              82| 1063498.0|
|              81|1053472.79|
|              80| 991767.38|
+----------------+----------+
only showing top 20 rows

--- 1.143153190612793 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False

In [19]:
# Write out a csv file from Spark
df.write.csv('out_home_sales_revised.csv', mode='overwrite')